- lstm-gru with predicted ids by lstm-gru
- gpt2 max
- base uncased f1, 2
- base CASED f4, 5
- large uncased 5 f0
- large CASED 2 f1

めっちゃ速くなってる？もっかい投げる

なんか速くなってるからもう一個アンサンブルする

In [ ]:
%%time
!cd ../input/apex-master/apex-master/apex-master/ && pip install --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .

In [ ]:
import shutil
import sys
# copy our file into the working directory
sys.path.insert(0, "../input/pytorch-pretrained-bert-2/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master/")

# LSTM-GRU bs-256 crawl ROOV TEST

In [ ]:
DEBUG = False

batch_size = 256

LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 2 * LSTM_UNITS
n_seeds = 1
n_splits = 10
n_epochs = 15
EMBED_SIZE = 300
SUBGROUP_NEGATIVE_WEIGHT_COEF = 1
BACKGROUND_POSITIVE_WEIGHT_COEF = 0

ENSEMBLE_START_EPOCH = 3

MAX_LEN = 300

EMB_DROPOUT = 0.3
MIDDLE_DROPOUT = 0.3

if DEBUG:
    DEBUG_DATA_SIZE = 1000

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
MODEL_PATH = "../input/lstmgru-bs256-crawl-no-appos-predicted-ids-r/results"
os.listdir(MODEL_PATH)[:10]

In [ ]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random
from tqdm import tqdm_notebook as tqdm
from contextlib import contextmanager
from fastprogress import master_bar, progress_bar
from keras.preprocessing import sequence
from keras import preprocessing
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

In [ ]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
import re
# これだと、'はembeddingに結構入ってるのに除外されちゃう。　よくないので ' だけ抜いた
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '\n', '\r']

def clean_text(x: str) -> str:
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' {} '.format(punct))
    return x

def clean_numbers(x):
    return re.sub('\d+', ' ', x)

if DEBUG:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=DEBUG_DATA_SIZE)
else:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

x_test = test['comment_text'].apply(lambda x: clean_text(x))
gc.collect()

In [ ]:
aux_col_list = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [ ]:
%%time
with open('../input/reducing-oov-of-crawl300d2m-no-appos-result/jigsaw-crawl-300d-2M.joblib', 'rb') as f:
    crawl_emb_dict = joblib.load(f)

## Reduce OOV

In [ ]:
import operator
from typing import Dict, List
def build_vocab(texts: pd.DataFrame) -> Dict[str, int]:
    """
    
    Parameters
    -----
    texts: pandas.Series
        question textの列
        
    Returns
    -----
    dict: 
        単語とカウント
    
    """
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in tqdm(sentences):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab: Dict[str, int], embeddings_index: Dict) -> List[str]:
    """
    Parameters
    -----
    vocab: dict
        単語とカウント
    embeddings_index: dict
        load_embedの出力
        
    Returns:
        list:
            embeddingsに入ってない単語
    """
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in tqdm(vocab.keys()):
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / float(len(vocab))))
    print('Found embeddings for  {:.2%} of all text'.format(float(nb_known_words) / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

from nltk.stem import PorterStemmer
p_stemmer = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
l_stemmer = LancasterStemmer()
from nltk.stem import SnowballStemmer
s_stemmer = SnowballStemmer("english")

import copy
def edits1(word):
    """
    wordの編集距離1の単語のリストを返す
    """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def known(words, embed): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in embed)

def spellcheck(word, word_rank_dict):
    return min(known(edits1(word), word_rank_dict), key=lambda w: word_rank_dict[w])


import unicodedata
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "",
                 "€": "e", "™": "tm", "√": " sqrt ", "×": "x",
                 "²": "2", "—": "-", "–": "-", "’": "'",
                 "_": "-", "`": "'", '“': '"', '”': '"',
                 '“': '"', "£": "e", '∞': 'infinity',
                 'θ': 'theta', '÷': '/', 'α': 'alpha',
                 '•': '.', 'à': 'a', '−': '-', 'β': 'beta',
                 '∅': '', '³': '3', 'π': 'pi', }
def process_stemmer(vocab, embed):
    oov_word_set = set()
    for word in tqdm(vocab.keys()):
        vector = embed.get(word, None)
        if vector is not None:
            continue

        vector = embed.get(word.lower(), None)
        if vector is not None:
            embed[word] = vector
            continue

        vector = embed.get(word.upper(), None)
        if vector is not None:
            embed[word] = vector
            continue

        vector = embed.get(word.capitalize(), None)
        if vector is not None:
            embed[word] = vector
            continue
            
        corr_word = punct_mapping.get(word, None)
        if corr_word is not None:
            vector = embed.get(corr_word, None)
            if vector is not None:
                embed[word] = vector
                continue
        
        try:
            vector = embed.get(p_stemmer.stem(word), None)
        except:
            vector = embed.get(p_stemmer.stem(word.decode('utf-8')), None)
        if vector is not None:
            embed[word] = vector
            continue
            
        try:
            vector = embed.get(l_stemmer.stem(word), None)
        except:
            vector = embed.get(l_stemmer.stem(word.decode('utf-8')), None)
        if vector is not None:
            embed[word] = vector
            continue
        
        try:
            vector = embed.get(s_stemmer.stem(word), None)
        except:
            vector = embed.get(s_stemmer.stem(word.decode('utf-8')), None)     
        if vector is not None:
            embed[word] = vector
            continue
        
        oov_word_set.add(word)
            
    return embed, oov_word_set

def process_small_capital(vocab, embed, oov_set):
    oov_word_set = set()
    for word in tqdm(vocab.keys()):
        if word not in oov_set:
            continue
        
        char_list = []
        any_small_capitial = False
        for char in word:
            try:
                uni_name = unicodedata.name(char)
            except ValueError:
                continue

            if 'LATIN LETTER SMALL CAPITAL' in uni_name:
                char = uni_name[-1]
                any_small_capitial = True
            if 'CYRILLIC SMALL LETTER GHE WITH STROKE' in uni_name:
                char = 'F'
                any_small_capitial = True

            char_list.append(char)

        if not any_small_capitial:
            oov_word_set.add(word)
            continue
            
        legit_word = ''.join(char_list)
        
        vector = embed.get(legit_word, None)
        if vector is not None:
            embed[word] = vector
            continue

        vector = embed.get(legit_word.lower(), None)
        if vector is not None:
            embed[word] = vector
            continue

        vector = embed.get(legit_word.upper(), None)
        if vector is not None:
            embed[word] = vector
            continue

        vector = embed.get(legit_word.capitalize(), None)
        if vector is not None:
            embed[word] = vector
            continue
            
        corr_word = punct_mapping.get(legit_word, None)
        if corr_word is not None:
            vector = embed.get(corr_word, None)
            if vector is not None:
                embed[word] = vector
                continue
        
        try:
            vector = embed.get(p_stemmer.stem(legit_word), None)
        except:
            vector = embed.get(p_stemmer.stem(legit_word.decode('utf-8')), None)
        if vector is not None:
            embed[word] = vector
            continue
            
        try:
            vector = embed.get(l_stemmer.stem(legit_word), None)
        except:
            vector = embed.get(l_stemmer.stem(legit_word.decode('utf-8')), None)
        if vector is not None:
            embed[word] = vector
            continue
        
        try:
            vector = embed.get(s_stemmer.stem(legit_word), None)
        except:
            vector = embed.get(s_stemmer.stem(legit_word.decode('utf-8')), None)
                    
        if vector is not None:
            embed[word] = vector
            continue

        oov_word_set.add(word)
        
    return embed, oov_word_set

def process_spellcheck(vocab, embed, word_rank_dict, oov_set):
    oov_word_set = set()
    
    for word in tqdm(vocab.keys()):
        if word not in oov_set:
            continue
            
        try:
            vector = embed.get(spellcheck(word, word_rank_dict), None)
        except:
            continue
        if vector is not None:
            embed[word] = vector
            continue
            
        oov_word_set.add(word)
            
    return embed, oov_word_set

def make_word_rank(embed):
    word_rank = {}
    for i, word in enumerate(embed):
        word_rank[word] = i
    return word_rank

def head(enumerable, n=10):
    for i, item in enumerate(enumerable):
        print(str(i) + '\n', item)
        if i > n:
            return


In [ ]:
%%time
vocab = build_vocab(x_test)
oov = check_coverage(vocab, crawl_emb_dict)
head(oov)

In [ ]:
%%time
emb, oov_stemmer = process_stemmer(vocab, crawl_emb_dict)
oov = check_coverage(vocab, emb)
head(oov)

In [ ]:
%%time
emb, oov_small_capital = process_small_capital(vocab, emb, oov_stemmer)
oov = check_coverage(vocab, emb)
head(oov)

In [ ]:
%%time
word_rank = make_word_rank(emb)
emb, oov_spell = process_spellcheck(vocab, emb, word_rank, oov_small_capital)
oov = check_coverage(vocab, emb)
head(oov)

## Sentence features

In [ ]:
with open('../input/googleprofanitywords/list.txt', 'r') as f:
    p_words = f.readlines()
    
set_puncts = set(puncts)

p_word_set = set([t.replace('\n', '') for t in p_words])

def sentence_fetures(text):
    word_list = text.split()
    word_count = len(word_list)
    n_upper = len([word for word in word_list if any([c.isupper() for c in word])])
    n_unique = len(set(word_list))
    n_ex = word_list.count('!')
    n_que = word_list.count('?')
    n_puncts = len([word for word in word_list if word in set_puncts])
    n_prof = len([word for word in word_list if word in p_word_set])
    n_oov = len([word for word in word_list if word not in crawl_emb_dict])
    
    return word_count, n_upper, n_unique, n_ex, n_que, n_puncts, n_prof, n_oov

sentence_feature_cols = ['word_count', 'n_upper', 'n_unique', 'n_ex', 'n_que', 'n_puncts', 'n_prof', 'n_oov']
from collections import defaultdict
feature_dict = defaultdict(list)
for text in progress_bar(x_test):
    feature_list = sentence_fetures(text)
    for i_feature, feature_name in enumerate(sentence_feature_cols):
        feature_dict[sentence_feature_cols[i_feature]].append(feature_list[i_feature])
        
sentence_df = pd.DataFrame.from_dict(feature_dict)
for col in ['n_upper', 'n_unique', 'n_ex', 'n_que', 'n_puncts', 'n_prof', 'n_oov']:
    sentence_df[col + '_ratio'] = sentence_df[col] / sentence_df['word_count']
sentence_df.head()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
tmp = sentence_df.hist(ax=ax)

In [ ]:
sentence_feature_mat = sentence_df.drop(columns=['n_oov', 'n_oov_ratio']).values
gc.collect()

In [ ]:
%%time

x_test = x_test.apply(lambda x: clean_numbers(x))

tokenizer = preprocessing.text.Tokenizer(filters="", lower=False)
tokenizer.fit_on_texts(list(x_test))

x_test = tokenizer.texts_to_sequences(x_test)

max_features = len(tokenizer.word_index) + 1
max_features

def build_matrix(word_index, embedding_dict):
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_dict[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

emb_mat_crawl, oov = build_matrix(tokenizer.word_index, emb)
del crawl_emb_dict, oov, emb
gc.collect()

emb_mat_torch = torch.tensor(emb_mat_crawl, dtype=torch.float32).cuda()
del emb_mat_crawl
gc.collect()


In [ ]:
import math
class DynamicBucketIterator(object):
    def __init__(self, data, label, capacity, pad_token, shuffle, length_quantile, max_batch_size, for_bert):
        self.data = data
        self.label = label
        self.pad_token = pad_token
        self.capacity = capacity
        self.shuffle = shuffle
        self.length_quantile = length_quantile
        self.for_bert = for_bert
        
        self.index_sorted = sorted(range(len(self.data)), key=lambda i: len(self.data[i]))
        
        old_separator_index = 0
        self.separator_index_list = [0]
        for i_sample in range(len(self.data)):
            sample_index = self.index_sorted[i_sample]
            sample = self.data[sample_index]
            current_batch_size = i_sample - old_separator_index + 1
            if min(len(sample), MAX_LEN) * current_batch_size <= self.capacity and current_batch_size <= max_batch_size:
                pass
            else:
                old_separator_index = i_sample
                self.separator_index_list.append(i_sample)
                
        self.separator_index_list.append(len(self.data)) # [0, ..., start_separator_index, end_separator_index, ..., len(data)]
        
        if not self.shuffle:
            self.bucket_index = range(self.__len__())
        
        self.reset_index()

    def reset_index(self):
        self.i_batch = 0
        
        if self.shuffle:
            self.index_sorted = sorted(np.random.permutation(len(self.data)), key=lambda i: len(self.data[i]))
            self.bucket_index = np.random.permutation(self.__len__())
    
    def __len__(self):
        return len(self.separator_index_list) - 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            i_bucket = self.bucket_index[self.i_batch]
        except IndexError as e:
            self.reset_index()
            raise StopIteration
            
        start_index, end_index = self.separator_index_list[i_bucket : i_bucket + 2]
        
        index_batch = self.index_sorted[start_index : end_index]

        raw_batch_data = [self.data[i] for i in index_batch]
        
        batch_label = self.label[index_batch]
        
        math.ceil(1)
        
        max_len = int(math.ceil(np.quantile([len(x) for x in raw_batch_data], self.length_quantile)))
        max_len = min([max_len, MAX_LEN])
        if max_len == 0:
            max_len = 1
        
        if self.for_bert:
            segment_id_batch = np.zeros((len(raw_batch_data), max_len))
            padded_batch = []
            input_mask_batch = []
            for sample in raw_batch_data:
                input_mask = [1] * len(sample) + [0] * (max_len - len(sample))
                input_mask_batch.append(input_mask[:max_len])

                sample = sample + [self.pad_token for _ in range(max_len - len(sample))]
                padded_batch.append(sample[:max_len])

            self.i_batch += 1

            return padded_batch, segment_id_batch, input_mask_batch, batch_label, index_batch
        
        else:
            padded_batch = []
            for sample in raw_batch_data:
                sample = sample + [self.pad_token for _ in range(max_len - len(sample))]
                padded_batch.append(sample[:max_len])

            self.i_batch += 1

            return padded_batch, batch_label, index_batch

In [ ]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x
    
class GRULayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRULayer, self).__init__()
        
        self.gru = nn.GRU(input_size=input_size,
                          hidden_size=hidden_size,
                          bias=True,
                          bidirectional=True,
                          batch_first=True)
        
        self.init_weights()
        
    def init_weights(self):
        ih = (param.data for name, param in self.named_parameters() if 'weight_ih' in name)
        hh = (param.data for name, param in self.named_parameters() if 'weight_hh' in name)
        b = (param.data for name, param in self.named_parameters() if 'bias' in name)
        for k in ih:
            nn.init.xavier_uniform_(k)
        for k in hh:
            nn.init.orthogonal_(k)
        for k in b:
            nn.init.constant_(k, 0)

    def forward(self, x):
        gru_outputs, gru_state = self.gru(x)
        return gru_outputs, gru_state
    
    
class NeuralNet(nn.Module):
    def __init__(self, embed_size, num_aux_targets, num_sentence_features):
        super(NeuralNet, self).__init__()
        self.embedding_dropout = SpatialDropout(EMB_DROPOUT)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.middle_dropout = SpatialDropout(MIDDLE_DROPOUT)
        
        self.lstm2 = GRULayer(LSTM_UNITS * 2, LSTM_UNITS)
        
        self.linear_sentence1 = nn.Linear(num_sentence_features, num_sentence_features)
        
        n_hidden = LSTM_UNITS * 2 + num_sentence_features
        self.linear1 = nn.Linear(n_hidden, n_hidden) # bi-max, bi-ave, sentence_features
            
        self.linear_out = nn.Linear(n_hidden, 1)
        self.linear_aux_out = nn.Linear(n_hidden, num_aux_targets)
        
    def forward(self, x, sentence_features):
        h_embedding = self.embedding_dropout(x)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        
        h_lstm1 = self.middle_dropout(h_lstm1)
        
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
            
        h_sentence = self.linear_sentence1(sentence_features)
        h_cat = torch.cat((max_pool, h_sentence), 1)
        
        h_conc_linear1  = F.relu(self.linear1(h_cat))
        
        hidden = h_cat + h_conc_linear1
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

OOF_TRAIN_COL = 'oof_train'
SUBGROUP_AUC_COL = 'subgroup_auc'
BPSN_AUC_COL = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC_COL = 'bnsp_auc'  # stands for background negative, subgroup positive
from sklearn import metrics
def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan
'model'
def compute_subgroup_auc(df, subgroup_col, label_col, oof_col):
    subgroup_examples = df[df[subgroup_col]]
    return compute_auc(subgroup_examples[label_col], subgroup_examples[oof_col])

def compute_bpsn_auc(df, subgroup_col, label_col, oof_col):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup_col] & ~df[label_col]]
    non_subgroup_positive_examples = df[~df[subgroup_col] & df[label_col]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label_col], examples[oof_col])

def compute_bnsp_auc(df, subgroup_col, label_col, oof_col):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup_col] & df[label_col]]
    non_subgroup_negative_examples = df[~df[subgroup_col] & ~df[label_col]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label_col], examples[oof_col])

def compute_bias_metrics_for_model(df,
                                   subgroup_list,
                                   oof_col,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    record_list = []
    for subgroup in subgroup_list:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(df[df[subgroup]])
        }
        record[SUBGROUP_AUC_COL] = compute_subgroup_auc(df, subgroup, label_col, oof_col)
        record[BPSN_AUC_COL] = compute_bpsn_auc(df, subgroup, label_col, oof_col)
        record[BNSP_AUC_COL] = compute_bnsp_auc(df, subgroup, label_col, oof_col)
        record_list.append(record)
    return pd.DataFrame(record_list).sort_values('subgroup_auc', ascending=True)

TOXICITY_COLUMN = 'target'
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
# Convert taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC_COL], POWER),
        power_mean(bias_df[BPSN_AUC_COL], POWER),
        power_mean(bias_df[BNSP_AUC_COL], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

def get_various_auc(valid_df, y_pred):
    valid_df = convert_dataframe_to_bool(valid_df.fillna(0))
    valid_df.loc[:, OOF_TRAIN_COL] = y_pred
    valid_df = convert_dataframe_to_bool(valid_df.fillna(0))
    bias_metrics_df = compute_bias_metrics_for_model(valid_df, identity_columns, OOF_TRAIN_COL, TOXICITY_COLUMN)
    overall_auc = calculate_overall_auc(valid_df, OOF_TRAIN_COL)
    return get_final_metric(bias_metrics_df, overall_auc), overall_auc, bias_metrics_df

In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
from sklearn.preprocessing import StandardScaler
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

auc_array = np.zeros((n_seeds, n_splits, n_epochs))

test_array = np.zeros((n_seeds, n_splits, n_epochs, len(x_test)))

from sklearn.model_selection import KFold
kf = KFold(n_splits=n_splits, shuffle=True)
for i_seed in range(n_seeds):
    print(f'start seed {i_seed}')

    for i_fold, (dev_index, val_index) in enumerate(kf.split(x_test)): # 使ってるのは i_foldだけ
        print(f'start fold {i_fold}')
        scaler = joblib.load(os.path.join(MODEL_PATH, f'scaler-seed{i_seed}-fold{i_fold}.joblib'))
        test_sentence_feature_mat = scaler.transform(sentence_feature_mat)

        test_loader = DynamicBucketIterator(x_test, 
                                            torch.tensor(np.concatenate([np.zeros((len(x_test), 1+len(aux_col_list))),
                                                         test_sentence_feature_mat], axis=1),dtype=torch.float32).cuda(),
                                            capacity=MAX_LEN*batch_size, pad_token=0, shuffle=False,
                                            length_quantile=1, max_batch_size=10000000, for_bert=False)
        
        print(torch.cuda.memory_allocated())
        for i_epoch in range(ENSEMBLE_START_EPOCH, n_epochs):
            start_time = time.time()
            model = NeuralNet(EMBED_SIZE, len(aux_col_list), sentence_feature_mat.shape[-1])
            load_path = os.path.join(MODEL_PATH, f'seed{i_seed}-fold{i_fold}-epoch{i_epoch}.torchModelState')
            model.load_state_dict(torch.load(load_path))
            model.cuda()
            
            model.eval()
            
            epoch_test_pred = np.zeros(len(x_test))
            mem_list = []
            for batch in test_loader:
                x_batch = batch[0]
                y_batch = batch[1]
                index_batch = batch[2]
                
                y_true_batch = y_batch[:, :1+len(aux_col_list)]
                sample_weight_batch = y_batch[:, 1+len(aux_col_list)]
                sentence_feature_batch = y_batch[:, -sentence_feature_mat.shape[-1]:]
                
                if len(x_batch) >= 32:
                    y_pred = model(emb_mat_torch[x_batch], sentence_feature_batch)
                else:
                    # イミフだけど、len(x_batch)が32未満だとバグる
                    len_last = len(x_batch)
                    x_batch.extend([[0 for _ in range(len(x_batch[0]))] for _ in range(32)])
                    y_pred = model(emb_mat_torch[x_batch[:32]][:len_last], sentence_feature_batch)
                
                mem_list.append(torch.cuda.memory_allocated())
                epoch_test_pred[index_batch] = sigmoid(y_pred[:, 0].detach().cpu().numpy())
                
                del y_pred
                torch.cuda.empty_cache()
                            
            test_array[i_seed, i_fold, i_epoch] = epoch_test_pred
            
            elapsed_time = time.time() - start_time
            
            del model
            gc.collect()
            torch.cuda.empty_cache()
            print(f'epoch {i_epoch}: {elapsed_time: 0.3f}, {max(mem_list)/10**9}')
            

        del epoch_test_pred
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
test_pred_rnn = np.mean(test_array[:, :, ENSEMBLE_START_EPOCH:], axis=(0, 1, 2))
plt.hist(test_pred_rnn, bins=20)

In [ ]:
del test_loader, emb_mat_torch, test_array
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

# GPT-2 max

In [ ]:
i_seed = 0
    
SENTENCE_FEAURE_USED = ['word_count', 'n_unique', 'n_unique_ratio']

batch_size = 32

OUT_DROPOUT = 0.3

BERT_HIDDEN_SIZE = 768

MAX_LEN = 220

In [ ]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random
from tqdm import tqdm
tqdm.pandas()
from contextlib import contextmanager
from fastprogress import master_bar, progress_bar
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sentence_feature_mat = sentence_df[SENTENCE_FEAURE_USED].values
gc.collect()

In [ ]:
%%time
if DEBUG:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=DEBUG_DATA_SIZE)
else:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

x_test = test['comment_text']
del test
gc.collect()

In [ ]:
import torch
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
tokenizer = GPT2Tokenizer(vocab_file='../input/gpt2-models/vocab.json', merges_file='../input/gpt2-models/merges.txt')

In [ ]:
%%time
x_test = x_test.progress_apply(lambda x: tokenizer.encode(x[:MAX_LEN]))

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_aux_targets, num_sentence_features):
        super(NeuralNet, self).__init__()
        self.gpt2 = GPT2Model.from_pretrained('../input/gpt2-models/')
        self.dropout = nn.Dropout(OUT_DROPOUT)
        
        self.linear_sentence1 = nn.Linear(num_sentence_features, num_sentence_features)
        
        n_hidden = BERT_HIDDEN_SIZE + num_sentence_features
        self.linear1 = nn.Linear(n_hidden, n_hidden)
        
        self.linear_out = nn.Linear(n_hidden, 1)
        self.linear_aux_out = nn.Linear(n_hidden, num_aux_targets)
        
    def forward(self, x_features, sentence_features):
        
        h_gpt2, _ = self.gpt2(x_features) # (bsz, seq_len, hidden_size)
        
        h_gpt2, _ = torch.max(h_gpt2, 1) # (bsz, hidden_size)
        
        h_sentence = self.linear_sentence1(sentence_features)
        
        h_cat = torch.cat((h_gpt2, h_sentence), 1)
        
        h_conc_linear1  = F.relu(self.linear1(h_cat))
        
        hidden = h_cat + h_conc_linear1
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [ ]:
MODEL_PATH = '../input/gpt2-f1e1-max-results'
os.listdir(MODEL_PATH)

In [ ]:
from sklearn.preprocessing import StandardScaler
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

aux_col_list = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

test_dict = {}
fold_list = [1]
i_epoch = 1
start_time = time.time()
for i_fold in fold_list:
    fold_start_time = time.time()
    
    print(f'start fold {i_fold}')
    scaler = joblib.load(os.path.join(MODEL_PATH, f'scaler-seed{0}-fold{i_fold}.joblib'))
    test_sentence_feature_mat = scaler.transform(sentence_feature_mat)

    test_loader = DynamicBucketIterator(x_test, 
                                        torch.tensor(np.concatenate([np.zeros((len(x_test), 1+len(aux_col_list))), test_sentence_feature_mat], axis=1),dtype=torch.float32).cuda(),
                                        capacity=MAX_LEN*batch_size, pad_token=0, shuffle=False, length_quantile=1, max_batch_size=2048, for_bert=False)

    print(torch.cuda.memory_allocated())

    model = NeuralNet(len(aux_col_list), sentence_feature_mat.shape[-1])
    load_path = os.path.join(MODEL_PATH, f'seed{0}-fold{i_fold}-epoch{i_epoch}.torchModelState')
    model.load_state_dict(torch.load(load_path)['model_state_dict'])
    model.cuda()

    model.eval()

    epoch_test_pred = np.zeros(len(x_test))
    eval_start_time = time.time()
    for batch in progress_bar(test_loader):
        x_batch = torch.tensor(batch[0], dtype=torch.long).cuda()
        y_batch = batch[1]
        index_batch = batch[2]
        
        y_true_batch = y_batch[:, :1+len(aux_col_list)]
        sample_weight_batch = y_batch[:, 1+len(aux_col_list)]
        sentence_feature_batch = y_batch[:, -sentence_feature_mat.shape[-1]:]

        y_pred = model(x_batch, sentence_feature_batch)
    #                 print('after_prediction', torch.cuda.memory_allocated())
        epoch_test_pred[index_batch] = sigmoid(y_pred[:, 0].detach().cpu().numpy())
        del x_batch, y_pred
        torch.cuda.empty_cache()
    print(f'uncased {i_fold} finishd in {time.time() - fold_start_time}', file=open('time.txt', 'a'))

    test_dict[i_fold] = epoch_test_pred
    del model, epoch_test_pred
    gc.collect()
    torch.cuda.empty_cache()
    
print(f'uncased finishd in {time.time() - start_time}', file=open('time.txt', 'a'))
    


In [ ]:
test_pred_gpt2_max = np.mean(list(test_dict.values()), axis=0)

plt.hist(test_pred_gpt2_max, bins=20)



In [ ]:
del test_loader, test_dict
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

# base uncased f1, 2

In [ ]:
i_seed = 0
    
SENTENCE_FEAURE_USED = ['word_count', 'n_unique', 'n_unique_ratio']

batch_size = 32

OUT_DROPOUT = 0.3

BERT_HIDDEN_SIZE = 768

BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
BERT_DO_LOWER = 'uncased' in BERT_MODEL_PATH

MAX_LEN = 220


In [ ]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random
from contextlib import contextmanager
from fastprogress import master_bar, progress_bar
from keras.preprocessing import sequence
from keras import preprocessing
from tqdm import tqdm
tqdm.pandas()
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

In [ ]:
%%time
if DEBUG:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=DEBUG_DATA_SIZE)
else:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

x_test = test['comment_text']
del test
gc.collect()

In [ ]:
sentence_df.head(10)

In [ ]:
sentence_feature_mat = sentence_df[SENTENCE_FEAURE_USED].values
gc.collect()

In [ ]:
WORK_DIR = "../working/"
os.listdir(WORK_DIR)

In [ ]:
import torch
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)


In [ ]:
BERT_DO_LOWER

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    BERT_MODEL_PATH, cache_dir=None, do_lower_case=BERT_DO_LOWER)

In [ ]:
def tokenize(text, max_len, tokenizer):
    tokenized_text = tokenizer.tokenize(text)[:max_len-2]
    return ["[CLS]"]+tokenized_text+["[SEP]"]

In [ ]:
%%time
# cleanしたのは x_test_cleanedに入ってて、x_test はそのまま
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
x_test = x_test.progress_apply(lambda x: tokenize(x, MAX_LEN, tokenizer))

In [ ]:
max(len(t) for t in x_test)

In [ ]:
x_test.head(20)

In [ ]:
x_test = x_test.progress_apply(lambda x: tokenizer.convert_tokens_to_ids(x))

In [ ]:
x_test.head(20)

In [ ]:
del tokenizer
gc.collect()

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
with suppress_stdout():
    convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
        BERT_MODEL_PATH + 'bert_model.ckpt',
    BERT_MODEL_PATH + 'bert_config.json',
    WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
from glob import glob
def get_model_path(i_seed, i_fold, i_epoch, base_path):
    for dir_name in glob(base_path + '*'):
        if (i_seed == int(re.match('.*s\d', dir_name).group()[-1]) and
            i_fold == int(re.match('.*f\d', dir_name).group()[-1]) and
            i_epoch == int(re.match('.*e\d', dir_name).group()[-1])
           ):
            return dir_name
            

In [ ]:
get_model_path(0, 0, 1, '../input/base-uncased')

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_aux_targets, num_sentence_features):
        super(NeuralNet, self).__init__()
        self.bert = BertModel.from_pretrained(WORK_DIR)
        self.dropout = nn.Dropout(OUT_DROPOUT)
        
        self.linear_sentence1 = nn.Linear(num_sentence_features, num_sentence_features)
        
        n_hidden = BERT_HIDDEN_SIZE + num_sentence_features
        self.linear1 = nn.Linear(n_hidden, n_hidden)
        
        self.linear_out = nn.Linear(n_hidden, 1)
        self.linear_aux_out = nn.Linear(n_hidden, num_aux_targets)
        
    def forward(self, x_features, sentence_features):
        
        _, bert_output = self.bert(*x_features, output_all_encoded_layers=False)
        
        bert_output = self.dropout(bert_output)
        
        h_sentence = self.linear_sentence1(sentence_features)
        
        h_cat = torch.cat((bert_output, h_sentence), 1)
        
        h_conc_linear1  = F.relu(self.linear1(h_cat))
        
        hidden = h_cat + h_conc_linear1
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [ ]:
from sklearn.preprocessing import StandardScaler
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

aux_col_list = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

test_dict = {}
fold_list = [0]

i_epoch = 2
start_time = time.time()
for i_fold in fold_list:
    fold_start_time = time.time()
    MODEL_PATH = '../input/bert-base-uncased-ftpe18-e2-5e6'
    print(f'start fold {i_fold}')
    scaler = joblib.load(os.path.join(MODEL_PATH, f'scaler-seed{0}-fold{i_fold}.joblib'))
    test_sentence_feature_mat = scaler.transform(sentence_feature_mat)

    test_loader = DynamicBucketIterator(x_test, 
                                        torch.tensor(np.concatenate([np.zeros((len(x_test), 1+len(aux_col_list))), test_sentence_feature_mat], axis=1),dtype=torch.float32).cuda(),
                                        capacity=MAX_LEN*batch_size, pad_token=0, shuffle=False, length_quantile=1, max_batch_size=2048, for_bert=True)

    print(torch.cuda.memory_allocated())

    model = NeuralNet(len(aux_col_list), sentence_feature_mat.shape[-1])
    load_path = os.path.join(MODEL_PATH, f'seed{0}-fold{i_fold}-epoch{i_epoch}.torchModelState')
    model.load_state_dict(torch.load(load_path)['model_state_dict'])
    model.cuda()

    model.eval()

    epoch_test_pred = np.zeros(len(x_test))
    eval_start_time = time.time()
    for batch in test_loader:
        x_batch = batch[0]
        segment_id_batch = batch[1]
        input_mask_batch = batch[2]
        y_batch = batch[3]
        index_batch = batch[4]
        y_true_batch = y_batch[:, :1+len(aux_col_list)]
        sample_weight_batch = y_batch[:, 1+len(aux_col_list)]
        sentence_feature_batch = y_batch[:, -sentence_feature_mat.shape[-1]:]
        x_features = [torch.tensor(feature, dtype=torch.long).cuda() for feature in [x_batch, segment_id_batch, input_mask_batch]]
    #                 print('x_features', torch.cuda.memory_allocated())

        y_pred = model(x_features, sentence_feature_batch)
    #                 print('after_prediction', torch.cuda.memory_allocated())
        epoch_test_pred[index_batch] = sigmoid(y_pred[:, 0].detach().cpu().numpy())
        del x_features, y_pred
        torch.cuda.empty_cache()
    print(f'uncased {i_fold} finishd in {time.time() - fold_start_time}', file=open('time.txt', 'a'))

    test_dict[i_fold] = epoch_test_pred
    del model, epoch_test_pred
    gc.collect()
    torch.cuda.empty_cache()
    
print(f'uncased finishd in {time.time() - start_time}', file=open('time.txt', 'a'))

In [ ]:
test_pred_base_uncased_ftp13 = np.mean(list(test_dict.values()), axis=0)

plt.hist(test_pred_base_uncased_ftp13, bins=20)


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_aux_targets, num_sentence_features):
        super(NeuralNet, self).__init__()
        self.bert_model = BertModel.from_pretrained(WORK_DIR)
        self.dropout = nn.Dropout(OUT_DROPOUT)
        
        self.linear_sentence1 = nn.Linear(num_sentence_features, num_sentence_features)
        
        n_hidden = BERT_HIDDEN_SIZE + num_sentence_features
        self.linear1 = nn.Linear(n_hidden, n_hidden)
        
        self.linear_out = nn.Linear(n_hidden, 1)
        self.linear_aux_out = nn.Linear(n_hidden, num_aux_targets)
        
    def forward(self, x_features, sentence_features):
        
        _, bert_output = self.bert_model(*x_features, output_all_encoded_layers=False)
        
        bert_output = self.dropout(bert_output)
        
        h_sentence = self.linear_sentence1(sentence_features)
        
        h_cat = torch.cat((bert_output, h_sentence), 1)
        
        h_conc_linear1  = F.relu(self.linear1(h_cat))
        
        hidden = h_cat + h_conc_linear1
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [ ]:
from sklearn.preprocessing import StandardScaler
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

aux_col_list = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

test_dict = {}
fold_list = [1, 2]

i_epoch = 1
start_time = time.time()
for i_fold in fold_list:
    fold_start_time = time.time()
    MODEL_PATH = get_model_path(i_seed, i_fold, i_epoch, '../input/base-uncased')
    print(f'start fold {i_fold}')
    scaler = joblib.load(os.path.join(MODEL_PATH, f'scaler-seed{0}-fold{i_fold}.joblib'))
    test_sentence_feature_mat = scaler.transform(sentence_feature_mat)

    test_loader = DynamicBucketIterator(x_test, 
                                        torch.tensor(np.concatenate([np.zeros((len(x_test), 1+len(aux_col_list))), test_sentence_feature_mat], axis=1),dtype=torch.float32).cuda(),
                                        capacity=MAX_LEN*batch_size, pad_token=0, shuffle=False, length_quantile=1, max_batch_size=2048, for_bert=True)

    print(torch.cuda.memory_allocated())

    model = NeuralNet(len(aux_col_list), sentence_feature_mat.shape[-1])
    load_path = os.path.join(MODEL_PATH, f'seed{0}-fold{i_fold}-epoch{i_epoch}.torchModelState')
    model.load_state_dict(torch.load(load_path)['model_state_dict'])
    model.cuda()

    model.eval()

    epoch_test_pred = np.zeros(len(x_test))
    eval_start_time = time.time()
    for batch in test_loader:
        x_batch = batch[0]
        segment_id_batch = batch[1]
        input_mask_batch = batch[2]
        y_batch = batch[3]
        index_batch = batch[4]
        y_true_batch = y_batch[:, :1+len(aux_col_list)]
        sample_weight_batch = y_batch[:, 1+len(aux_col_list)]
        sentence_feature_batch = y_batch[:, -sentence_feature_mat.shape[-1]:]
        x_features = [torch.tensor(feature, dtype=torch.long).cuda() for feature in [x_batch, segment_id_batch, input_mask_batch]]
    #                 print('x_features', torch.cuda.memory_allocated())

        y_pred = model(x_features, sentence_feature_batch)
    #                 print('after_prediction', torch.cuda.memory_allocated())
        epoch_test_pred[index_batch] = sigmoid(y_pred[:, 0].detach().cpu().numpy())
        del x_features, y_pred
        torch.cuda.empty_cache()
    print(f'uncased {i_fold} finishd in {time.time() - fold_start_time}', file=open('time.txt', 'a'))

    test_dict[i_fold] = epoch_test_pred
    del model, epoch_test_pred
    gc.collect()
    torch.cuda.empty_cache()
    
print(f'uncased finishd in {time.time() - start_time}', file=open('time.txt', 'a'))
    


In [ ]:
test_pred_base_uncased = np.mean(list(test_dict.values()), axis=0)

plt.hist(test_pred_base_uncased, bins=20)

del test_loader, test_dict
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()


# base CASED f4, 5

In [ ]:
fold_list = [4, 5]

i_seed = 0
    
SENTENCE_FEAURE_USED = ['word_count', 'n_unique', 'n_unique_ratio']

batch_size = 32

OUT_DROPOUT = 0.3

BERT_HIDDEN_SIZE = 768

BERT_MODEL_PATH = '../input/bert-pretrained-models/cased_l-12_h-768_a-12/cased_L-12_H-768_A-12/'
BERT_DO_LOWER = 'uncased' in BERT_MODEL_PATH

MAX_LEN = 220

In [ ]:
if DEBUG:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=DEBUG_DATA_SIZE)
else:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

x_test = test['comment_text']
del test
gc.collect()

In [ ]:
BERT_DO_LOWER

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    BERT_MODEL_PATH, cache_dir=None, do_lower_case=BERT_DO_LOWER)

In [ ]:
def tokenize(text, max_len, tokenizer):
    tokenized_text = tokenizer.tokenize(text)[:max_len-2]
    return ["[CLS]"]+tokenized_text+["[SEP]"]
x_test = x_test.progress_apply(lambda x: tokenize(x, MAX_LEN, tokenizer))

In [ ]:
max(len(t) for t in x_test)

In [ ]:
x_test.head(20)

In [ ]:
x_test = x_test.progress_apply(lambda x: tokenizer.convert_tokens_to_ids(x))

In [ ]:
x_test.head(20)

In [ ]:
del tokenizer
gc.collect()

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
with suppress_stdout():
    convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
        BERT_MODEL_PATH + 'bert_model.ckpt',
    BERT_MODEL_PATH + 'bert_config.json',
    WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
get_model_path(i_seed, 4, 1, '../input/base-cased')

In [ ]:
from sklearn.preprocessing import StandardScaler
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

test_dict = {}

i_epoch = 1
start_time = time.time()
for i_fold in fold_list:
    fold_start_time = time.time()
    MODEL_PATH = get_model_path(i_seed, i_fold, i_epoch, '../input/base-cased')
    print(f'start fold {i_fold}')
    scaler = joblib.load(os.path.join(MODEL_PATH, f'scaler-seed{0}-fold{i_fold}.joblib'))
    test_sentence_feature_mat = scaler.transform(sentence_feature_mat)

    test_loader = DynamicBucketIterator(x_test, 
                                        torch.tensor(np.concatenate([np.zeros((len(x_test), 1+len(aux_col_list))), test_sentence_feature_mat], axis=1),dtype=torch.float32).cuda(),
                                        capacity=MAX_LEN*batch_size, pad_token=0, shuffle=False, length_quantile=1, max_batch_size=2048, for_bert=True)

    print(torch.cuda.memory_allocated())

    model = NeuralNet(len(aux_col_list), sentence_feature_mat.shape[-1])
    load_path = os.path.join(MODEL_PATH, f'seed{0}-fold{i_fold}-epoch{i_epoch}.torchModelState')
    model.load_state_dict(torch.load(load_path)['model_state_dict'])
    model.cuda()

    model.eval()

    epoch_test_pred = np.zeros(len(x_test))
    eval_start_time = time.time()
    for batch in test_loader:
        x_batch = batch[0]
        segment_id_batch = batch[1]
        input_mask_batch = batch[2]
        y_batch = batch[3]
        index_batch = batch[4]
        y_true_batch = y_batch[:, :1+len(aux_col_list)]
        sample_weight_batch = y_batch[:, 1+len(aux_col_list)]
        sentence_feature_batch = y_batch[:, -sentence_feature_mat.shape[-1]:]
        x_features = [torch.tensor(feature, dtype=torch.long).cuda() for feature in [x_batch, segment_id_batch, input_mask_batch]]
    #                 print('x_features', torch.cuda.memory_allocated())
    #                 timer.stamp(f'x_features')

        y_pred = model(x_features, sentence_feature_batch)
    #                 print('after_prediction', torch.cuda.memory_allocated())
    #                 timer.stamp(f'after_prediction')
        epoch_test_pred[index_batch] = sigmoid(y_pred[:, 0].detach().cpu().numpy())
        del x_features, y_pred
        torch.cuda.empty_cache()
    print(f'uncased {i_fold} finishd in {time.time() - fold_start_time}', file=open('time.txt', 'a'))

    test_dict[i_fold] = epoch_test_pred
    del model, epoch_test_pred
    gc.collect()
    torch.cuda.empty_cache()
    
print(f'uncased finishd in {time.time() - start_time}', file=open('time.txt', 'a'))
    

In [ ]:
test_pred_base_CASED = np.mean(list(test_dict.values()), axis=0)

In [ ]:
del test_loader, test_dict
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

# Large uncased f0e1

In [ ]:
batch_size = 8
n_seeds = 1
n_splits = 10
n_epochs = 1

TRAIN_ON_N_SPLITS = 1

EMBED_SIZE = 300
SUBGROUP_NEGATIVE_WEIGHT_COEF = 1
BACKGROUND_POSITIVE_WEIGHT_COEF = 0

BERT_HIDDEN_SIZE = 1024

BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'
BERT_DO_LOWER = 'uncased' in BERT_MODEL_PATH

ENSEMBLE_START_EPOCH = 0

OUT_DROPOUT = 0.3

MAX_LEN = 220
    


In [ ]:
MODEL_PATH = "../input/bert-large-5"
os.listdir(MODEL_PATH)

In [ ]:
%%time

import re
# これだと、'はembeddingに結構入ってるのに除外されちゃう。　よくないので ' だけ抜いた
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '\n', '\r']

def clean_text(x: str) -> str:
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' {} '.format(punct))
    return x

def clean_numbers(x):
    return re.sub('\d+', ' ', x)

if DEBUG:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=DEBUG_DATA_SIZE)
else:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

x_test_clean = test['comment_text'].apply(lambda x: clean_text(x)).apply(lambda x: clean_numbers(x))
del test

gc.collect()

x_test_clean.head(20)

def add_cls_sep(text):
    return "[CLS] " + text + " [SEP]"

x_test = x_test_clean.apply(lambda x: add_cls_sep(x))

x_test.head(10)


In [ ]:
BERT_DO_LOWER

In [ ]:

tokenizer = BertTokenizer.from_pretrained(
    BERT_MODEL_PATH, cache_dir=None, do_lower_case=BERT_DO_LOWER)

tqdm.pandas()
x_test = x_test.progress_apply(lambda x: tokenizer.tokenize(x))

x_test.head(10)

In [ ]:
x_test = x_test.progress_apply(lambda x: tokenizer.convert_tokens_to_ids(x)[:MAX_LEN])

x_test.head(10)

In [ ]:
del tokenizer
gc.collect()

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
with suppress_stdout():
    convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
        BERT_MODEL_PATH + 'bert_model.ckpt',
    BERT_MODEL_PATH + 'bert_config.json',
    WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_aux_targets, num_sentence_features):
        super(NeuralNet, self).__init__()
        self.bert_model = BertModel.from_pretrained(WORK_DIR)
        self.dropout = nn.Dropout(OUT_DROPOUT)
        
        self.linear_sentence1 = nn.Linear(num_sentence_features, num_sentence_features)
        
        n_hidden = BERT_HIDDEN_SIZE + num_sentence_features
        self.linear1 = nn.Linear(n_hidden, n_hidden)
        
        self.linear_out = nn.Linear(n_hidden, 1)
        self.linear_aux_out = nn.Linear(n_hidden, num_aux_targets)
        
    def forward(self, x_features, sentence_features):
        
        _, bert_output = self.bert_model(*x_features, output_all_encoded_layers=False)
        
        bert_output = self.dropout(bert_output)
        
        h_sentence = self.linear_sentence1(sentence_features)
        
        h_cat = torch.cat((bert_output, h_sentence), 1)
        
        h_conc_linear1  = F.relu(self.linear1(h_cat))
        
        hidden = h_cat + h_conc_linear1
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out
        

In [ ]:
aux_col_list = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [ ]:
sentence_feature_mat = sentence_df.drop(columns=['n_oov']).values


In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
from sklearn.preprocessing import StandardScaler
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

test_array = np.zeros(len(x_test))

from sklearn.model_selection import KFold
kf = KFold(n_splits=n_splits, shuffle=True)
for i_seed in range(n_seeds):
    print(f'start seed {i_seed}')

    for i_fold, (dev_index, val_index) in enumerate(kf.split(x_test)): # 使ってるのは i_foldだけ
        if i_fold >= TRAIN_ON_N_SPLITS:
            break
            
        print(f'start fold {i_fold}')
        scaler = joblib.load(os.path.join(MODEL_PATH, f'scaler-seed{0}-fold{0}.joblib'))
        test_sentence_feature_mat = scaler.transform(sentence_feature_mat)

        test_loader = DynamicBucketIterator(x_test, 
                                            torch.tensor(np.concatenate([np.zeros((len(x_test), 1+len(aux_col_list))),
                                                                         test_sentence_feature_mat], axis=1),dtype=torch.float32).cuda(),
                                            capacity=MAX_LEN*batch_size, pad_token=0, shuffle=False, length_quantile=1, max_batch_size=2048, for_bert=True)
        
        print(torch.cuda.memory_allocated())
        mb = master_bar(range(n_epochs))

        i_epoch = 1
        
        model = NeuralNet(len(aux_col_list), sentence_feature_mat.shape[-1])
        load_path = os.path.join(MODEL_PATH, f'seed{0}-fold{0}-epoch{i_epoch*4+3}.torchModelState')
        model.load_state_dict(torch.load(load_path))
        model.cuda()
        print(i_epoch)
        start_time = time.time()

        model.eval()

        epoch_test_pred = np.zeros(len(x_test))
        for batch in progress_bar(test_loader):
            x_batch = batch[0]
            segment_id_batch = batch[1]
            input_mask_batch = batch[2]
            y_batch = batch[3]
            index_batch = batch[4]
            y_true_batch = y_batch[:, :1+len(aux_col_list)]
            sample_weight_batch = y_batch[:, 1+len(aux_col_list)]
            sentence_feature_batch = y_batch[:, -sentence_feature_mat.shape[-1]:]

            x_features = [torch.tensor(feature, dtype=torch.long).cuda() for feature in [x_batch, segment_id_batch, input_mask_batch]]
#                 print('x_features', torch.cuda.memory_allocated())
#                 timer.stamp(f'x_features')
            y_pred = model(x_features, sentence_feature_batch)
#                 print('after_prediction', torch.cuda.memory_allocated())
#                 timer.stamp(f'after_prediction')

            epoch_test_pred[index_batch] = sigmoid(y_pred[:, 0].detach().cpu().numpy())
            del x_features, y_pred
            torch.cuda.empty_cache()

        test_array = epoch_test_pred
        elapsed_time = time.time() - start_time
        del model, epoch_test_pred
        gc.collect()
        torch.cuda.empty_cache()


In [ ]:
test_pred_large_uncased = test_array

In [ ]:
plt.hist(test_pred_large_uncased, bins=20)

In [ ]:
del test_loader
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()


# BERT Large CASED f0

In [ ]:
batch_size = 8
n_seeds = 1
n_splits = 10
n_epochs = 1

TRAIN_ON_N_SPLITS = 1

EMBED_SIZE = 300
SUBGROUP_NEGATIVE_WEIGHT_COEF = 1
BACKGROUND_POSITIVE_WEIGHT_COEF = 0

BERT_HIDDEN_SIZE = 1024

BERT_MODEL_PATH = '../input/bert-pretrained-models/cased_l-24_h-1024_a-16/cased_L-24_H-1024_A-16/'
BERT_DO_LOWER = 'uncased' in BERT_MODEL_PATH

ENSEMBLE_START_EPOCH = 0

OUT_DROPOUT = 0.3

MAX_LEN = 220
    


In [ ]:
MODEL_PATH = "../input/large-cased-1"
os.listdir(MODEL_PATH)

In [ ]:
%%time

import re
# これだと、'はembeddingに結構入ってるのに除外されちゃう。　よくないので ' だけ抜いた
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '\n', '\r']

def clean_text(x: str) -> str:
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' {} '.format(punct))
    return x

def clean_numbers(x):
    return re.sub('\d+', ' ', x)

if DEBUG:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=DEBUG_DATA_SIZE)
else:
    test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

x_test_clean = test['comment_text'].apply(lambda x: clean_text(x)).apply(lambda x: clean_numbers(x))
del test

gc.collect()

x_test_clean.head(20)

def add_cls_sep(text):
    return "[CLS] " + text + " [SEP]"

x_test = x_test_clean.apply(lambda x: add_cls_sep(x))

x_test.head(10)


In [ ]:
BERT_DO_LOWER

In [ ]:

tokenizer = BertTokenizer.from_pretrained(
    BERT_MODEL_PATH, cache_dir=None, do_lower_case=BERT_DO_LOWER)

tqdm.pandas()
x_test = x_test.progress_apply(lambda x: tokenizer.tokenize(x))

x_test.head(10)

In [ ]:
x_test = x_test.progress_apply(lambda x: tokenizer.convert_tokens_to_ids(x)[:MAX_LEN])

x_test.head(10)

In [ ]:
del tokenizer
gc.collect()

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
with suppress_stdout():
    convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
        BERT_MODEL_PATH + 'bert_model.ckpt',
    BERT_MODEL_PATH + 'bert_config.json',
    WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_aux_targets, num_sentence_features):
        super(NeuralNet, self).__init__()
        self.bert_model = BertModel.from_pretrained(WORK_DIR)
        self.dropout = nn.Dropout(OUT_DROPOUT)
        
        self.linear_sentence1 = nn.Linear(num_sentence_features, num_sentence_features)
        
        n_hidden = BERT_HIDDEN_SIZE + num_sentence_features
        self.linear1 = nn.Linear(n_hidden, n_hidden)
        
        self.linear_out = nn.Linear(n_hidden, 1)
        self.linear_aux_out = nn.Linear(n_hidden, num_aux_targets)
        
    def forward(self, x_features, sentence_features):
        
        _, bert_output = self.bert_model(*x_features, output_all_encoded_layers=False)
        
        bert_output = self.dropout(bert_output)
        
        h_sentence = self.linear_sentence1(sentence_features)
        
        h_cat = torch.cat((bert_output, h_sentence), 1)
        
        h_conc_linear1  = F.relu(self.linear1(h_cat))
        
        hidden = h_cat + h_conc_linear1
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out
        

In [ ]:
aux_col_list = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [ ]:
sentence_feature_mat = sentence_df.drop(columns=['n_oov']).values


In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

In [ ]:
from sklearn.preprocessing import StandardScaler
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

test_array = np.zeros(len(x_test))

from sklearn.model_selection import KFold
kf = KFold(n_splits=n_splits, shuffle=True)
for i_seed in range(n_seeds):
    print(f'start seed {i_seed}')

    for i_fold, (dev_index, val_index) in enumerate(kf.split(x_test)): # 使ってるのは i_foldだけ
        if i_fold >= TRAIN_ON_N_SPLITS:
            break
            
        print(f'start fold {i_fold}')
        scaler = joblib.load(os.path.join(MODEL_PATH, f'scaler-seed{0}-fold{0}.joblib'))
        test_sentence_feature_mat = scaler.transform(sentence_feature_mat)

        test_loader = DynamicBucketIterator(x_test, 
                                            torch.tensor(np.concatenate([np.zeros((len(x_test), 1+len(aux_col_list))),
                                                                         test_sentence_feature_mat], axis=1),dtype=torch.float32).cuda(),
                                            capacity=MAX_LEN*batch_size, pad_token=0, shuffle=False, length_quantile=1, max_batch_size=2048, for_bert=True)
        
        print(torch.cuda.memory_allocated())
        mb = master_bar(range(n_epochs))

        i_epoch = 1
        
        model = NeuralNet(len(aux_col_list), sentence_feature_mat.shape[-1])
        load_path = os.path.join(MODEL_PATH, f'seed{0}-fold{0}-epoch{i_epoch*4+3}.torchModelState')
        model.load_state_dict(torch.load(load_path))
        model.cuda()
        print(i_epoch)
        start_time = time.time()

        model.eval()

        epoch_test_pred = np.zeros(len(x_test))
        for batch in progress_bar(test_loader):
            x_batch = batch[0]
            segment_id_batch = batch[1]
            input_mask_batch = batch[2]
            y_batch = batch[3]
            index_batch = batch[4]
            y_true_batch = y_batch[:, :1+len(aux_col_list)]
            sample_weight_batch = y_batch[:, 1+len(aux_col_list)]
            sentence_feature_batch = y_batch[:, -sentence_feature_mat.shape[-1]:]

            x_features = [torch.tensor(feature, dtype=torch.long).cuda() for feature in [x_batch, segment_id_batch, input_mask_batch]]
#                 print('x_features', torch.cuda.memory_allocated())
#                 timer.stamp(f'x_features')
            y_pred = model(x_features, sentence_feature_batch)
#                 print('after_prediction', torch.cuda.memory_allocated())
#                 timer.stamp(f'after_prediction')

            epoch_test_pred[index_batch] = sigmoid(y_pred[:, 0].detach().cpu().numpy())
            del x_features, y_pred
            torch.cuda.empty_cache()

        test_array = epoch_test_pred
        elapsed_time = time.time() - start_time
        del model, epoch_test_pred
        gc.collect()
        torch.cuda.empty_cache()


In [ ]:
test_pred_large_CASED = test_array

In [ ]:
plt.hist(test_pred_large_CASED, bins=20)

In [ ]:
pred_list = [test_pred_rnn,
             test_pred_large_uncased, test_pred_large_CASED,
             test_pred_base_uncased_ftp13,
             test_pred_base_uncased, test_pred_base_CASED,
             test_pred_gpt2_max]

corr_mat = np.corrcoef(pred_list)
print(corr_mat)

import seaborn as sns
sns.heatmap(corr_mat, cmap='viridis')

test_pred = np.average(pred_list,
                       weights=[0.5, 2, 2, 1, 1.5, 1.5, 0.5], axis=0)


In [ ]:
plt.hist(test_pred, bins=20)

In [ ]:
df_submit = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
df_submit.prediction = test_pred
df_submit.to_csv('submission.csv', index=False)